# Hello World

In [12]:
%%javascript
// Import eCharts, the Javascript charting library
require.config({
    paths: {
        ech: 'https://cdnjs.cloudflare.com/ajax/libs/echarts/4.8.0/echarts.min'
    }
});

<IPython.core.display.Javascript object>

In [13]:
import json
import pandas as pd
import numpy as np

COLUMN = "retail_and_recreation_percent_change_from_baseline"
DATA_LOCATION = "<PATH TO FILES>"

def get_data():
    # Get the raw data
    raw_data = pd.read_csv("{}Global_Mobility_Report.csv".format(DATA_LOCATION))
    
    # Take only metrics by state (no sub regions). 
    # Note that we only want the records where sub_region_1 is not available
    raw_data = raw_data.fillna("NA")
    raw_data = raw_data.loc[raw_data["sub_region_1"] == 'NA']

    # Drop the columns that are not needed
    raw_data = raw_data.drop(["country_region_code", "sub_region_1", "sub_region_2"], axis=1)    
    
    # Create the JSON we will use in the Javascript code
    countries = np.unique(raw_data["country_region"])
    final_dict = {
        "dates": list(raw_data.loc[raw_data["country_region"]=="Afghanistan"].sort_values("date")["date"].values),
        "country_data":[]
    
    }
    
    # Create one element of the "country_data" array for each country
    for _c in countries:
        _tmp_data = raw_data.loc[raw_data["country_region"] == _c].sort_values("date")
        _data = _tmp_data[COLUMN]
        final_dict["country_data"].append({
            "name": _c,
            "data": list(_data.values)
        })

    # Return the JSON dumpe
    return json.dumps(final_dict)

In [14]:
%%javascript
// The output of get_data will be stored in the following function
window.outputVar = null

IPython.notebook.kernel.execute(
    "get_data()", 
    {
        iopub: {
            output: function(response) {
                // Get output as plain text
                var output = response.content.data["text/plain"];
                                console.log(output)
                
                // Remove unwanted characters
                output = output.substring(1, output.length-1).replace("\\'","'")
                
                // Set the variable
                window.outputVar = JSON.parse(output)
            }
        }
    },
    {
        silent: false, 
        store_history: false, 
        stop_on_error: true
    }
)


<IPython.core.display.Javascript object>

In [15]:
%%javascript
window.buildChart = function(cellElement){
(function(element) {
    // Execute the following code after requiring `ech`
    require(['ech'], function(ech) {
        
        // The `ech` variable contains echarts
        var echarts = ech
        
        // Append to the current "Cell Element" the chart's container
        cellElement.append("<div id='main' style='height:500px;'>hi</div>")
        
        // Init Highcharts
        var myChart = echarts.init(document.getElementById('main'));
        
        // Create data object
        var data = window.outputVar
        
        // Save the dates in a separate variable
        var dates = data.dates
        
        var series = []
        for(var _i=0; _i<data.country_data.length; _i++){
            var _name = data.country_data[_i]["name"]
            var _data = data.country_data[_i]["data"]
            
            series.push({
                name: _name,
                type: 'line',
                data: _data
            })
        }
        
        
        myChart.setOption({
            title: {
                text: 'Google COVID-19 Mobility'
            },
            tooltip: {
                trigger: 'item'
            },
            grid: {
                left: '3%',
                right: '4%',
                bottom: '3%',
                containLabel: true
            },
            xAxis: {
                type: 'category',
                data: dates // We can get this array directly from the `get_data()` output
            },
            yAxis: {
                type: 'value'
            },
            series: series // We just need to create this
        })
    })
})(element);
}

<IPython.core.display.Javascript object>

In [16]:
%%javascript
window.buildChart(element)

<IPython.core.display.Javascript object>

# Add events

In [7]:
import json
import pandas as pd
import numpy as np

COLUMNS = ["retail_and_recreation_percent_change_from_baseline",
           "grocery_and_pharmacy_percent_change_from_baseline",
           "parks_percent_change_from_baseline",
           "transit_stations_percent_change_from_baseline",
           "workplaces_percent_change_from_baseline",
           "residential_percent_change_from_baseline"]
DATA_LOCATION = "/Users/anialent/Projects/notebook-javascript/data/"

def get_data_all():
    # Get the raw data
    raw_data = pd.read_csv("{}Global_Mobility_Report.csv".format(DATA_LOCATION))
    
    # Take only metrics by state (no sub regions). 
    # Note that we only want the records where sub_region_1 is not available
    raw_data = raw_data.fillna("NA")
    raw_data = raw_data.loc[raw_data["sub_region_1"] == 'NA']

    # Drop the columns that are not needed
    raw_data = raw_data.drop(["country_region_code", "sub_region_1", "sub_region_2"], axis=1)    
    
    # Create the JSON we will use in the Javascript code
    countries = np.unique(raw_data["country_region"])
    final_dict = {
        "dates": list(raw_data.loc[raw_data["country_region"]=="Afghanistan"].sort_values("date")["date"].values),
        "country_data":[]
    
    }
    
    # Create one element of the "country_data" array for each country
    for _c in countries:
        _tmp_data = raw_data.loc[raw_data["country_region"] == _c].sort_values("date")
        
        # Create the new element in the final dict for the country data
        final_dict["country_data"].append({
            "name": _c
        })
        
        # Create one property for each KPI
        for _col in COLUMNS:
            _data = _tmp_data[_col]
            final_dict["country_data"][len(final_dict["country_data"])-1][_col] = list(_data.values)

    # Return the JSON dumpe
    return json.dumps(final_dict)


In [8]:
%%javascript
// The output of get_data will be stored in the following function
window.outputVarAll = null

IPython.notebook.kernel.execute(
    "get_data_all()", 
    {
        iopub: {
            output: function(response) {
                // Get output as plain text
                var output = response.content.data["text/plain"];
                                console.log(output)
                
                // Remove unwanted characters
                output = output.substring(1, output.length-1).replace("\\'","'")
                
                // Set the variable
                window.outputVarAll = JSON.parse(output)
            }
        }
    },
    {
        silent: false, 
        store_history: false, 
        stop_on_error: true
    }
)


<IPython.core.display.Javascript object>

In [9]:
%%javascript
// This time we have to pass some default paramters (countries selection and column to display)
window.buildChartSelect = function(cellElement, filteredCountries=[],
                                    column="retail_and_recreation_percent_change_from_baseline"){
    (function(element) {
        // Execute the following code after requiring `ech`
        require(['ech'], function(ech) {
            
            // The `ech` variable contains echarts
            var echarts = ech
            
            // Append to the current "Cell Element" the chart's container
            cellElement.append("<div id='main2' style='height:500px;'>hi</div>")
            
            // Init Highcharts
            var myChart = echarts.init(document.getElementById('main2'));

            // Create data object
            var data = window.outputVarAll
            
            // Save the dates in a separate variable
            var dates = data.dates
            
            // Generate the series according to the filters
            var series = []
            for(var _i=0; _i<data.country_data.length; _i++){
                if((filteredCountries.length > 0 && filteredCountries.indexOf(data.country_data[_i]["name"])>=0) 
                   || filteredCountries.length === 0){
                    // the `series object` will only contain the selected countries (all countries by default)
                    var _name = data.country_data[_i]["name"] 
                    
                    // the series object will contain only the selected column
                    var _data = data.country_data[_i][column] 

                    series.push({
                        name: _name,
                        type: 'line',
                        data: _data
                    })
                }
            }

            myChart.setOption({
                title: {
                    text: 'Google COVID-19 Mobility'
                },
                tooltip: {
                    trigger: 'item'
                },
                grid: {
                    left: '3%',
                    right: '4%',
                    bottom: '3%',
                    containLabel: true
                },
                xAxis: {
                    type: 'category',
                    data: dates
                },
                yAxis: {
                    type: 'value'
                },
                series: series
            })
        })
    })(element);
}

window.controlsOnChange = function(cellElement){
    // Get the selected countries
    var select = document.getElementById("countries");
    console.log(select)
    var selected = [];
    for (var i = 0; i < select.length; i++) {
        if (select.options[i].selected) selected.push(select.options[i].value);
    }
    
    // Get the selected KPI
    var kpis = document.getElementById("kpis");
    var column = kpis.options[kpis.selectedIndex].value;
    console.log(column)
    // Remove previous visual
    document.getElementById("main2").remove()
    window.buildChartSelect(cellElement, selected, column)
}

window.buildControls = function(cellElement){
    var data = window.outputVar
    
    // Generate the `select` element for the countries
    var controlHTML = document.createElement("select");
    controlHTML.setAttribute("style", "height:150px;");
    controlHTML.setAttribute("id", "countries");
    controlHTML.setAttribute("multiple", "multiple");
    
    // Generate the inner HTML
    var controlInnerHTML = ""
    for(var _i=0; _i<data.country_data.length; _i++){
        var _name = data.country_data[_i]["name"]
        controlInnerHTML += `<option value="${_name}">${_name}</option>`
    }
    // Set the inner HTML
    controlHTML.innerHTML = controlInnerHTML
    
    // !!! Enable events on the HTML object: on input change, we want to refresh the chart
    controlHTML.addEventListener('change', function() {
      window.controlsOnChange(cellElement)
    });
    
    // Append the new HTML object to the cell
    cellElement.append(controlHTML)
    
    // Generate the `select` elment to control which KPI to display
    var COLUMNS = ["retail_and_recreation_percent_change_from_baseline",
           "grocery_and_pharmacy_percent_change_from_baseline",
           "parks_percent_change_from_baseline",
           "transit_stations_percent_change_from_baseline",
           "workplaces_percent_change_from_baseline",
           "residential_percent_change_from_baseline"]
    
    // Generate the HTML element
    var controlKPIHTML = document.createElement("select");
    controlKPIHTML.setAttribute("id", "kpis");
    
    // Generate the options
    var controlKPIInnerHTML = ""
    for(var _i=0; _i<COLUMNS.length; _i++){
        var _name = COLUMNS[_i]
        controlKPIInnerHTML += `<option value="${_name}">${_name}</option>`
    }
    controlKPIHTML.innerHTML = controlKPIInnerHTML
    
    // !!! Enable events on the HTML object: on input change, we want to refresh the chart
    controlKPIHTML.addEventListener('change', function() {
      window.controlsOnChange(cellElement)
    });
    cellElement.append(controlKPIHTML)
}

<IPython.core.display.Javascript object>

In [10]:
%%javascript
window.buildChartSelect(element)
window.buildControls(element)


<IPython.core.display.Javascript object>